# Robo-MZOIP - ocjene prihvatljivosti za ekološku mrežu Natura 2000

## Uvod

### Natura 2000

Zakonski okvir: EU Direktiva o pticama (79/409/EEC), Direktiva o staništima (92/43/EEC)

http://ec.europa.eu/environment/nature/legislation/birdsdirective/index_en.htm

http://ec.europa.eu/environment/nature/legislation/habitatsdirective/index_en.htm

 * Područja očuvanja značajna za ptice
 * Područja očuvanja značajna za vrste i staništa

 * Postupak ocjene prihvatljivosti za ekološku mrežu
  - Prethodna ocjena
  - Glavna ocjena

## Problem

 * Nepregledna stranica Ministarstva zaštite okoliša i energetike.
 * Dokumenti se dodaju na način da je nemoguće sustavno pratiti njihovo dodavanje.

## Rješenje

 * Web scraper (python3 + requests + beautiful soup 4)
 * Twitter

In [136]:
import argparse
from datetime import datetime
import os
import re
import sys
import requests
from bs4 import BeautifulSoup
import pandas as pd

Definicija stranica koja nas zanimaju:

In [137]:
BASE_URL_MZOE = 'http://mzoe.hr'
BASE_URL_OPEM = 'http://mzoe.hr/hr/priroda/ekoloska-mreza-natura-2000.html'

Dohvaćanje html stranice, i kreiranje `soup` objekta sa stranice Ministarstva:

In [138]:
def get_soup(full_url):
    """Dohvaca soup objekt s mreznih stranica.

    Args:
        full_url (str): path do stranice.
    Returns:
        soup: beautifulsoup object.
    """
    r = requests.get(full_url)
    soup = BeautifulSoup(r.content, 'lxml')
    return soup

In [139]:
soup = get_soup(BASE_URL_OPEM)

### Glavne ocjene u nadležnosti Ministarstva

In [140]:
opem = soup.find('div', 'accordion').find_all('div', 'accordion')
a = opem[0].find_all('a')

In [141]:
print(a[0])
print(a[4])

<a href="/doc/16022018_-_informacija_o_zahtjevu_za_provodenje_postupka_glavne_ocjene_prihvatljivosti_za_ekolosku_mrezu.pdf">16.02.2018. - INFORMACIJA o zahtjevu za provođenje postupka Glavne ocjene prihvatljivosti za ekološku mrežu [PDF 630.39 kB]</a>
<a href="/doc/05122017_-_obavijest_o_javnoj_raspravi.pdf">05.12.2017. - OBAVIJEST o javnoj raspravi [PDF 777.69 kB]</a>


In [142]:
print(a[0].parent.parent.parent.previous_sibling.previous_sibling.get_text().strip())
print(BASE_URL_MZOE + a[0].get('href'))
print(a[4].parent.parent.parent.previous_sibling.previous_sibling.get_text().strip())
print(BASE_URL_MZOE + a[4].get('href'))

Uređenje vodnog puta rijeke Dunav kod Sotina na području Vukovarsko-srijemske županije
http://mzoe.hr/doc/16022018_-_informacija_o_zahtjevu_za_provodenje_postupka_glavne_ocjene_prihvatljivosti_za_ekolosku_mrezu.pdf
Radovi sanacije korita, uklanjanje nanosa i radovi redovitog održavanja vodnog režima na rijeci Bregani
http://mzoe.hr/doc/05122017_-_obavijest_o_javnoj_raspravi.pdf


In [143]:
opem_raw = []
for link in a:
    red = [link.parent.parent.parent.previous_sibling.previous_sibling.get_text().strip(),
           link.get_text(),
           BASE_URL_MZOE + link.get('href')]
    opem_raw.append(red)
opem_pd = pd.DataFrame(opem_raw)

In [144]:
opem_pd.head(10)

,0,1,2
0,Uređenje vodnog puta rijeke Dunav kod Sotina n...,16.02.2018. - INFORMACIJA o zahtjevu za provođ...,http://mzoe.hr/doc/16022018_-_informacija_o_za...
1,Uređenje vodnog puta rijeke Dunav kod Sotina n...,24.11.2015. - RJEŠENJE Ministarstva od 11. stu...,http://mzoe.hr/doc/24112015_-_rjesenje_minista...
2,Uređenje vodnog puta rijeke Dunav kod Sotina n...,02.10.2015. - INFORMACIJA o zahtjevu za provođ...,http://mzoe.hr/doc/02102015_-_informacija_o_za...
3,Izgradnja cestovnog mosta preko rijeke Drave n...,13.12.2017. - INFORMACIJA o zahtjevu za provođ...,http://mzoe.hr/doc/13122017_-_informacija_o_za...
4,"Radovi sanacije korita, uklanjanje nanosa i ra...",05.12.2017. - OBAVIJEST o javnoj raspravi [PDF...,http://mzoe.hr/doc/05122017_-_obavijest_o_javn...
5,"Radovi sanacije korita, uklanjanje nanosa i ra...",05.12.2017. - STUDIJA Glavne ocjene prihvatlji...,http://mzoe.hr/doc/05122017_-_studija_glavne_o...
6,"Radovi sanacije korita, uklanjanje nanosa i ra...",07.11.2017. - INFORMACIJA o zahtjevu za provođ...,http://mzoe.hr/doc/07112017_-_informacija_o_za...
7,Pregrada na rukavcu starog toka rijeke Drave,22.11.2017. - RJEŠENJE Ministarstva od 21. stu...,http://mzoe.hr/doc/22112017_-_rjesenje_minista...
8,Pregrada na rukavcu starog toka rijeke Drave,12.09.2017. - OBAVIJEST o javnoj raspravi [PDF...,http://mzoe.hr/doc/12092017_-_obavijest_o_javn...
9,Pregrada na rukavcu starog toka rijeke Drave,12.09.2017. - STUDIJA Glavne ocjene prihvatlji...,http://mzoe.hr/doc/12092017_-_studija_glavne_o...


### Glavne ocjene u nadležnosti županija/grada Zagreba

In [145]:
a1 = opem[1].find_all('a')
opem_z_raw = []
for link in a1:
    red = [link.parent.parent.parent.previous_sibling.previous_sibling.get_text().strip(),
           link.get_text(),
          BASE_URL_MZOE + link.get('href')]
    opem_z_raw.append(red)
opem_z_pd = pd.DataFrame(opem_z_raw)

In [146]:
opem_z_pd.head(10)

,0,1,2
0,Zaštita naselja Gradac i grada Pleternice od v...,18.10.2016. - OBAVIJEST o provedbi javne raspr...,http://mzoe.hr/doc/18102016_-_obavijest_o_prov...
1,Ekološka revitalizacija područja starog korita...,22.09.2016. - OBJAVA javne rasprave o Studiji ...,http://mzoe.hr/doc/22092016_-_objava_javne_ras...
2,Održavanje korita rijeke Dretulje na području ...,19.05.2016. - OBJAVA javne rasprave o Studiji ...,http://mzoe.hr/doc/19052016_-_objava_javne_ras...
3,Obnova desnog nasipa rijeke Save s obaloutvrda...,16.12.2015. - OBJAVA javne rasprave o Studiji ...,http://mzoe.hr/doc/16122015_-_objava_javne_ras...
4,Lječilišno-turistički kompleks 'Meline' u zalj...,05.11.2015. - OBJAVA javne rasprave o Studiji ...,http://mzoe.hr/doc/05112015_-_informacija_o_po...


### Prethodne ocjene prihvatljivosti za ekološku mrežu

In [147]:
prethodne = soup.find_all('div', 'accordion')[3]

In [148]:
godpattern = re.compile(r".*?(\d{4}).*")

In [149]:
prethodne_god = prethodne.find_all('h3')
prethodne_raw = []
for god in prethodne_god:
    dokumenti = god.next_sibling.next_sibling.find_all('a')
    for dok in dokumenti:
        red = [re.sub(godpattern,
                      "\\1",
                      god.get_text().strip()),
               dok.get_text(),
               BASE_URL_MZOE + dok.get('href')]
        prethodne_raw.append(red)
prethodne_pd = pd.DataFrame(prethodne_raw)

In [150]:
prethodne_pd.head(10)

,0,1,2
0,2018,02.03.2018. - Izgradnja peradarnika za uzgoj p...,http://mzoe.hr/doc/02032018_-_izgradnja_perada...
1,2018,27.02.2018. - Izgradnja drvene pješačke staze ...,http://mzoe.hr/doc/27022018_-_izgradnja_drvene...
2,2018,26.02.2018. - Prenamjena poljoprivrednog zemlj...,http://mzoe.hr/doc/26022018_-_prenamjena_poljo...
3,2018,21.02.2018. - Županijska razvojna strategija P...,http://mzoe.hr/doc/21022018_-_zupanijska_razvo...
4,2018,16.02.2018. - Autocesta A7 dionica između čvor...,http://mzoe.hr/doc/16022018_-_autocesta_a7_dio...
5,2018,"16.02.2018. - Uzgajalište školjaka u Blacama, ...",http://mzoe.hr/doc/16022018_-_uzgajaliste_skol...
6,2018,13.02.2018. - Rekonstrukcija (nadogradnja i pr...,http://mzoe.hr/doc/13022018_-_rekonstrukcija_n...
7,2018,09.02.2018. - III. Izmjene i dopune Prostornog...,http://mzoe.hr/doc/09022018_-_iii_izmjene_i_do...
8,2018,06.02.2018. - Prenamjena poljoprivrednog zemlj...,http://mzoe.hr/doc/06022018_-_prenamjena_poljo...
9,2018,06.02.2018. - Transformatorska stanica KTS 10(...,http://mzoe.hr/doc/06022018_-_transformatorska...


## To do

 * Snimanje u .tsv tablice
 * Integracija u `puobot.py`
 * Twitter upload